The purpose of this notebook is to create a function that takes the imported player list:
- clean to only needed information
- seperate by pither and batter
- merge each on player name to get projections from model
- merge back pitcher and batter data 
- output df for now

Next steps:
- build line up based on salary

In [1]:
# imports
import pandas as pd
import numpy as np

In [2]:
    # function
def build_lineup(df):
        # clean up fd to match column list above
        df.drop(columns=['First Name', 'Last Name', 'FPPG', 'Played', 'Injury Details', 'Tier', 'Batting Order', 'Roster Position'], inplace=True)

        # filling nulls for probable pitcher
        df['Probable Pitcher'].fillna('No', inplace=True)

        # fill nulls for injury indicator
        df['Injury Indicator'].fillna('Healthy', inplace=True)

        # renaming nickname column
        df.rename(columns={'Nickname': 'Name'}, inplace=True)

        # fitler to only healthy players
        df = df.loc[df['Injury Indicator'] == 'Healthy']

        # split using .loc by position and make new dataframe for pitchers
        pitchers = df.loc[df['Position']=='P']

        # split using .loc by position and make new dataframe for batters
        batters = df.loc[df['Position']!='P']

        # pitcher dataframe steps
        # save pitchers df to only starting pitchers
        pitchers = pitchers.loc[pitchers['Probable Pitcher']=='Yes']

        # read in pitcher projections
        pitcher_proj = pd.read_csv('../Projections/pitcher_projections_2021.csv')

        # merge attempt
        pitcher_projections = pitchers.merge(pitcher_proj, how='left', on='Name')

        # drop nulls if any
        pitcher_projections.dropna(inplace=True)

        # overwrite df with only the columns needed
        pitcher_projections = pitcher_projections[['Id', 'Position', 'Name', 'Salary', 'Team_x', 'Opponent', 'Projected_FPPG']]

        # rename team column
        pitcher_projections.rename(columns={'Team_x' : 'Team'}, inplace=True)

        # batter dataframe steps
        # read in projections file
        batter_21 = pd.read_csv('../Projections/batter_projections_2021.csv')

        # merge projections with batter df, creating new df
        batters_projections = batters.merge(batter_21, how='left', on='Name')

        # dropping batters with no projections
        batters_projections.dropna(inplace=True)
        
        #only batters with more than 40 ab's
        #batters_projections = batters_projections.loc[batters_projections['AB']>50]

        # drop unneeded columns for merge with pitcher, overwrite current df
        batters_projections = batters_projections[['Id', 'Position', 'Name', 'Salary', 'Team_x',
                                                   'Opponent','Projected_FPPG']]
        
        # rename team column
        batters_projections.rename(columns={'Team_x' : 'Team'}, inplace=True)

        # following are steps for creating a line up
        # set cap for fanduel
        salary_cap = 35_000

        # sort pitcher by fppg projections
        pitcher_projections.sort_values(by='Projected_FPPG', ascending=False, inplace=True, ignore_index=True)

        # create a player list to 
        lineup = []
        lineup.append(pitcher_projections.values[0])

        # need to update remaining salary
        salary_cap -= pitcher_projections['Salary'][0]

        # with updated salary fill remaining roster based on position and highest fppg
        # create position list for remaining roster spots
        position_list = ['C', '1B', '2B', '3B', 'SS', 'OF', 'OF', 'OF']
        # clean position
        pos_list = [pos[:2] for pos in batters_projections['Position']]
        batters_projections['Position'] = pos_list
        # sort batters by FPPG
        batters_projections.sort_values(by='Projected_FPPG', ascending=False, inplace=True, ignore_index=True)

        # create count based on remaining positions
        sal_count = 8

        # create average salary variable for remaining players
        avg_sal = salary_cap/sal_count

        # create for loop for each position in list to take highest fppg
        for pos in position_list:
            # setting counter to increase if player is already in list
            # this is inside the for loop beacuse it needs to be per position
            counter = 0
            # if salary greater than average move to next player
            for salary in batters_projections.loc[batters_projections['Position'] == pos]['Salary']:
                # test if salary is greater than average if it is increase counter
                if batters_projections.loc[batters_projections['Position'] == pos]['Salary'].values[counter] > avg_sal or batters_projections.loc[batters_projections['Position'] == pos]['Salary'].values[counter] > salary_cap:
                    counter += 1
                else:
                    # if less than average add player to list
                    lineup.append(batters_projections.loc[batters_projections['Position'] == pos].values[counter])
                    # drop player so no duplicates are added
                    batters_projections.drop(batters_projections.loc[batters_projections['Position'] == pos].index.values[counter], inplace=True)
                    # decrease sal_count
                    sal_count -= 1
                    # decrease salary cap
                    salary_cap -= batters_projections.loc[batters_projections['Position'] == pos]['Salary'].values[counter]
                    # create new average salary
                    avg_sal = salary_cap/sal_count
                    break
        # create dataframe of lineup
        df_lineup = pd.DataFrame(lineup, columns=['Id', 'Position', 'Name', 'Salary', 'Team', 'Opponent','Projected_FPPG'])

        # return dataframe
        return df_lineup

---

In [13]:
# test function
fd = pd.read_csv('../../../Downloads/FanDuel-MLB-2021 ET-07 ET-07 ET-61498-players-list.csv')

In [11]:
# overwrite imported data with dataframe from function
fd = build_lineup(fd)

<ipython-input-2-4e3af00bb648>:112: RuntimeWarning: divide by zero encountered in longlong_scalars
  avg_sal = salary_cap/sal_count


In [12]:
fd

,Id,Position,Name,Salary,Team,Opponent,Projected_FPPG
0,61498-102001,P,Yusei Kikuchi,9000,SEA,NYY,66.500471
1,61498-12988,C,Robinson Chirinos,2000,CHC,PHI,25.990382
2,61498-53401,1B,Trey Mancini,3300,BAL,TOR,9.009749
3,61498-13501,2B,Chris Owings,2500,COL,ARI,9.892806
4,61498-52147,3B,Kris Bryant,3500,CHC,PHI,9.373753
5,61498-12961,SS,Brandon Crawford,3300,SF,STL,9.871710
6,61498-82536,OF,Cedric Mullins,3400,BAL,TOR,9.675819
7,61498-102591,OF,Jake Fraley,2700,SEA,NYY,9.137190
8,61498-73159,OF,Mitch Haniger,3400,SEA,NYY,9.131983


In [58]:
# create function to create template for upload
def make_template(df):
    template = df[['Position', 'Id']]
    template = template.set_index('Position').T
    template.rename(columns={'C':'C/1B', '1B':'UTIL'}, inplace=True)
    return template

In [60]:
# test with dataframe above
temp = make_template(fd2)

In [61]:
#display template
temp

Position,P,C/1B,UTIL,2B,3B,SS,OF,OF,OF
Id,58417-68654,58417-79174,58417-52175,58417-68587,58417-5104,58417-37982,58417-5763,58417-60643,58417-39086


In [14]:
def build_lineup_stack(df):
        # clean up fd to match column list above
        df.drop(columns=['First Name', 'Last Name', 'FPPG', 'Played', 'Injury Details', 'Tier', 'Batting Order', 'Roster Position'], inplace=True)

        # filling nulls for probable pitcher
        df['Probable Pitcher'].fillna('No', inplace=True)

        # fill nulls for injury indicator
        df['Injury Indicator'].fillna('Healthy', inplace=True)

        # renaming nickname column
        df.rename(columns={'Nickname': 'Name'}, inplace=True)

        # fitler to only healthy players
        df = df.loc[df['Injury Indicator'] == 'Healthy']

        # split using .loc by position and make new dataframe for pitchers
        pitchers = df.loc[df['Position']=='P']

        # split using .loc by position and make new dataframe for batters
        batters = df.loc[df['Position']!='P']

        # pitcher dataframe steps
        # save pitchers df to only starting pitchers
        pitchers = pitchers.loc[pitchers['Probable Pitcher']=='Yes']

        # read in pitcher projections
        pitcher_proj = pd.read_csv('../Projections/pitcher_projections_2021.csv')

        # merge attempt
        pitcher_projections = pitchers.merge(pitcher_proj, how='left', on='Name')

        # drop nulls if any
        pitcher_projections.dropna(inplace=True)

        # overwrite df with only the columns needed
        pitcher_projections = pitcher_projections[['Id', 'Position', 'Name', 'Salary', 'Team_x', 'Opponent', 'AVG', 'Projected_FPPG']]

        # rename team column
        pitcher_projections.rename(columns={'Team_x' : 'Team'}, inplace=True)

        # batter dataframe steps
        # read in projections file
        batter_21 = pd.read_csv('../Projections/batter_projections_2021.csv')

        # merge projections with batter df, creating new df
        batters_projections = batters.merge(batter_21, how='left', on='Name')

        # dropping batters with no projections
        batters_projections.dropna(inplace=True)

        # drop unneeded columns for merge with pitcher, overwrite current df
        batters_projections = batters_projections[['Id', 'Position', 'Name', 'Salary', 'Team_x', 'Opponent','Projected_FPPG']]
        
        # rename team column
        batters_projections.rename(columns={'Team_x' : 'Team'}, inplace=True)

        # following are steps for creating a line up
        # set cap for fanduel
        salary_cap = 35_000
        
        # sort by avg
        pitcher_projections.sort_values(by='AVG', ascending=False, inplace=True, ignore_index=True)
        # create list of teams to filter batters
        team_list = []
        for x in range(0,4):
            team_list.append(pitcher_projections['Opponent'][x])
        # drop avg
        pitcher_projections.drop(columns='AVG', inplace=True)
        
        # create team filter 
        team_filter = (batters_projections['Team'] == team_list[0]) | (batters_projections['Team'] == team_list[1]) |(batters_projections['Team'] == team_list[2]) |(batters_projections['Team'] == team_list[3])
        # new batter dataframe with team filter
        batters_projections = batters_projections[team_filter]
        # reset index
        batters_projections.reset_index(drop=True, inplace=True)

        # sort pitcher by fppg projections
        pitcher_projections.sort_values(by='Projected_FPPG', ascending=False, inplace=True, ignore_index=True)

        # create a player list to 
        lineup = []
        lineup.append(pitcher_projections.values[0])

        # need to update remaining salary
        salary_cap -= pitcher_projections['Salary'][0]

        # with updated salary fill remaining roster based on position and highest fppg
        # create position list for remaining roster spots
        position_list = ['C', '1B', '2B', '3B', 'SS', 'OF', 'OF', 'OF']
        # clean position
        pos_list = [pos[:2] for pos in batters_projections['Position']]
        batters_projections['Position'] = pos_list
        # sort batters by FPPG
        batters_projections.sort_values(by='Projected_FPPG', ascending=False, inplace=True, ignore_index=True)

        # create count based on remaining positions
        sal_count = 8

        # create average salary variable for remaining players
        avg_sal = salary_cap/sal_count

        # create for loop for each position in list to take highest fppg
        for pos in position_list:
            # setting counter to increase if player is already in list
            # this is inside the for loop beacuse it needs to be per position
            counter = 0
            # if salary greater than average move to next player
            for salary in batters_projections.loc[batters_projections['Position'] == pos]['Salary']:
                # test if salary is greater than average if it is increase counter
                if batters_projections.loc[batters_projections['Position'] == pos]['Salary'].values[counter] > avg_sal or batters_projections.loc[batters_projections['Position'] == pos]['Salary'].values[counter] > salary_cap:
                    #print('y')
                    counter += 1
                else:
                    # if less than average add player to list
                    lineup.append(batters_projections.loc[batters_projections['Position'] == pos].values[counter])
                    # drop player so no duplicates are added
                    batters_projections.drop(batters_projections.loc[batters_projections['Position'] == pos].index.values[counter], inplace=True)
                    # decrease sal_count
                    sal_count -= 1
                    # decrease salary cap
                    salary_cap -= batters_projections.loc[batters_projections['Position'] == pos]['Salary'].values[counter]
                    # create new average salary
                    avg_sal = salary_cap/sal_count
                    break
        # create dataframe of lineup
        df_lineup = pd.DataFrame(lineup, columns=['Id', 'Position', 'Name', 'Salary', 'Team', 'Opponent','Projected_FPPG'])

        # return dataframe
        return df_lineup, pd.DataFrame(team_list)

In [15]:
fd4, team = build_lineup_stack(fd)
team

<ipython-input-14-36d787763152>:124: RuntimeWarning: divide by zero encountered in longlong_scalars
  avg_sal = salary_cap/sal_count


,0
0,TOR
1,ARI
2,PHI
3,SD


In [16]:
fd4

,Id,Position,Name,Salary,Team,Opponent,Projected_FPPG
0,61498-102001,P,Yusei Kikuchi,9000,SEA,NYY,66.500471
1,61498-17139,C,J.T. Realmuto,3000,PHI,CHC,8.103519
2,61498-118210,1B,Jake Cronenworth,3100,SD,WSH,8.757537
3,61498-13188,2B,Jean Segura,2800,PHI,CHC,7.983235
4,61498-13239,3B,Eduardo Escobar,3300,ARI,COL,8.753887
5,61498-82654,SS,Bo Bichette,3900,TOR,BAL,10.248423
6,61498-12948,OF,Bryce Harper,3800,PHI,CHC,9.766982
7,61498-5456,OF,Andrew McCutchen,2900,PHI,CHC,9.045121
8,61498-79175,OF,Trent Grisham,3700,SD,WSH,8.862743


In [18]:
fd4['Salary'].sum()

35500